In [10]:
from web3 import AsyncWeb3, Web3
from web3.eth import Contract
from datetime import datetime
from pprint import pprint

import json


def read_abi(token: str):
    with open(f"../config/{token.lower()}.json") as f:
        data = json.load(f)
        return data

In [11]:
w3 = Web3(Web3.HTTPProvider("https://bitter-wandering-feather.arbitrum-mainnet.quiknode.pro/862a558ad28be94cf6b1ccae509bdca74a19086a"))

strategy_account_address = Web3.to_checksum_address("0x04df99681dd2c0d26598139afd517142430b1202")

strategy_reserve_abi = read_abi("strategy-reserve")
strategy_reserve = w3.eth.contract(
    address=strategy_account_address,
    abi=strategy_reserve_abi,
)

bank = strategy_reserve.functions.STRATEGY_BANK().call()
print('bank', bank)


bank_abi = read_abi("strategy-bank")
strategy_bank = w3.eth.contract(
    address=bank,
    abi=bank_abi,
)

strategy_account_abi = read_abi("strategy-account")
strategy_account = w3.eth.contract(
    address=strategy_account_address,
    abi=strategy_account_abi,
)


bank 0x479889160FEECe9C0fB0FDDF3b45312f54D719CC


## Init ERC20 token contract


In [18]:
erc20_abi = read_abi("erc20_abi")
## Get strategy bank asset

asset = strategy_bank.functions.STRATEGY_ASSET().call()
print('asset', asset)

usdc_contract = w3.eth.contract(address=asset, abi=erc20_abi)


## Get strategy account holdings
balance = usdc_contract.functions.balanceOf(strategy_account_address).call()
print('balance', balance)

asset 0xaf88d065e77c8cC2239327C5EDb3A432268e5831
balance 1484407


In [13]:
holdings = strategy_bank.functions.getStrategyAccountHoldings(
    strategy_account_address,
).call()
pprint({
    'collateral': holdings[0],
    'loan': holdings[1],
    'interestIndexLast': holdings[2]
})

{'collateral': 187710256,
 'interestIndexLast': 28130407131623183,
 'loan': 360000000}


### Dung cai nay tinh health factor

In [25]:
holdings_after_pay_interest = strategy_bank.functions.getStrategyAccountHoldingsAfterPayingInterest(
    strategy_account_address,
).call()
pprint({
    'collateral': holdings_after_pay_interest[0],
    'loan': holdings_after_pay_interest[1],
    'interestIndexLast': holdings_after_pay_interest[2]
})

healthScore = (holdings_after_pay_interest[0]) / holdings_after_pay_interest[1]
print('healthScore', healthScore)

{'collateral': 187179934,
 'interestIndexLast': 29603521971370380,
 'loan': 360000000}
healthScore 0.5199442611111111


In [14]:
account_value = strategy_account.functions.getAccountValue().call()
print('account_value', account_value)

account_value 357509748


### Calculate health score

`loss = holding.loan`

`adjustedCollateral = holdings.collateral - Math.min(holdings.collateral, loss);`

In [23]:

healthScore = (holdings[0] - 3.16) / holdings[1]
print('healthScore', healthScore)



healthScore 0.521417369
